In [62]:
import requests
import os
import glob
from natsort import natsorted
import json
import csv
from datetime import datetime
import pandas as pd
import socket

Test Script

In [63]:
payload = {'statement': 'select 1;', 'pretty': 'true', 'client_context_id': 'xyz'}
response = requests.get('http://localhost:19002/query/service', params = payload)
response.json()
# row['metrics']['executionTime'] = float(row['metrics']['executionTime'].replace('ms', ''))
# row['metrics']['executionTime']

{'requestID': '54ef3bab-89de-454f-b677-57e50e931307',
 'clientContextID': 'xyz',
 'signature': {'*': '*'},
 'results': [{'$1': 1}],
 'plans': {},
 'status': 'success',
 'metrics': {'elapsedTime': '108.976463ms',
  'executionTime': '95.214164ms',
  'resultCount': 1,
  'resultSize': 15,
  'processedObjects': 0}}

In [64]:
isTiny = True

if isTiny:
    dataverse = "USE TinyBenchmark;"
else:
    dataverse = "USE BigBenchmark;"

dataverse += "\n"
print(dataverse)

USE TinyBenchmark;



In [65]:
cwd = os.getcwd()
cwd += "/"
cwd

'/Users/andretran/Documents/Projects/AsterixDB/'

In [66]:
addr = input("Enter address: ")

# print(ip)

downloadsPath = ''

if addr == '127.0.0.1':
    downloadsPath = cwd + "datasets/"
elif addr == '1':
    downloadsPath = "/local_data/downloads/datasets"
    
print(downloadsPath)

Enter address: 127.0.0.1
/Users/andretran/Documents/Projects/AsterixDB/datasets/


In [67]:
# path = '/Users/andretran/Documents/Projects/AsterixDB/'
sqlExt = "*.sql"
queriesPath = cwd + "queries/" + sqlExt
print(queriesPath)
tinyQueriesPath = cwd + "tinyQueries/" + sqlExt
bigQueriesPath = cwd + "bigQueries/" + sqlExt

queries = glob.glob(queriesPath)
tinyQueries = glob.glob(tinyQueriesPath)
bigQueries = glob.glob(bigQueriesPath)

if isTiny:
    benchmarkQueries = queries + tinyQueries
else:
    benchmarkQueries = queries + bigQueries

benchmarkQueries = natsorted(benchmarkQueries, key=os.path.basename)
print(benchmarkQueries)

/Users/andretran/Documents/Projects/AsterixDB/queries/*.sql
['/Users/andretran/Documents/Projects/AsterixDB/tinyQueries/1a_Create_All.sql', '/Users/andretran/Documents/Projects/AsterixDB/tinyQueries/2a_Load_All.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/3_Point-Count_Buildings.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/4_Point-Count_RoadNetwork.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/5_Point-Count_AllNodes.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/6_Equals_Polygon-Polygon.sql', '/Users/andretran/Documents/Projects/AsterixDB/tinyQueries/7a_Equals_Point-Point.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/8_Disjoint_Polygon-Polygon.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/9_Intersects_Line-Polygon.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/10_Intersects_Point-Polygon.sql', '/Users/andretran/Documents/Projects/AsterixDB/tinyQueries/11a_Intersects_Point-Line.sql',

In [68]:
# Import Module

# Folder Path
# path = "/Users/andretran/Documents/Projects/AsterixDB/queries"

# all_files = glob.glob('/Users/andretran/Documents/Projects/AsterixDB/queries/*.sql')

# Change the directory
os.chdir(cwd)
jsonFile = "tinybenchmark.json"
csvFile = "tinybenchmark.csv"

def create_json_file():
    if os.path.exists(jsonFile):
        os.remove(jsonFile)
        print('JSON file deleted')
    with open(jsonFile, mode='w', encoding='utf-8') as f:
        json.dump([], f)
        print('JSON file created')
        
def create_csv_file():
    if os.path.exists(csvFile):
        os.remove(csvFile)
        print('CSV file deleted')
    with open(csvFile, mode='w', encoding='utf-8') as f:
        header = ['requestID', 'clientContextID', 'status', 'elapsedTime', 'resultCount', 'resultSize', 'processedObjects', 'timestamp']
        writer = csv.writer(f)
        # write the header
        writer.writerow(header)
        print('CSV file created')
        
create_json_file()
create_csv_file()

def benchmark_queries(file_path, index):
    with open(file_path, 'r') as f:
        statement = f.read()
        if index != 0:
            statement = dataverse + statement
        if index == 22 or 1:
            statement = statement.replace('$PATH$', addr + '://' + downloadsPath)
            
        print(statement)
        payload = {'statement': statement, 'pretty': 'true', 'client_context_id': os.path.basename(file_path), 'readonly': False}
        timestamp = datetime.now()
        response = requests.post('http://localhost:19002/query/service', params = payload)
        row = response.json()
        print(row['status'])
        with open(jsonFile, "r") as of:
            data = json.load(of)
        data.append(row)
        with open(jsonFile, "w") as of:
            json.dump(data, of)
        with open(csvFile, mode='a', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([row['requestID'], row['clientContextID'], row['status'], row['metrics']['elapsedTime'], row['metrics']['resultCount'], row['metrics']['resultSize'], row['metrics']['processedObjects'], timestamp])


# iterate through all file
for index, file_path in enumerate(benchmarkQueries):
    # Check whether file is in text format or not
    print(os.path.basename(file_path))
    benchmark_queries(file_path, index)

print("done")

JSON file deleted
JSON file created
CSV file deleted
CSV file created
1a_Create_All.sql
DROP DATAVERSE TinyBenchmark IF EXISTS;
CREATE DATAVERSE TinyBenchmark;
USE TinyBenchmark;

CREATE TYPE RoadNetworkType AS {
	id: UUID,
	g: geometry,
	attr0: string?,
	attr1: string?,
	attr2: string?,
	attr3: string?,
	attr4: string?
};

CREATE TYPE BuildingsType AS {
	id: UUID,
	g: geometry,
	attr0: string?,
	attr1: string?
};

CREATE TYPE AllNodesType AS {
	id: UUID,
	g: geometry,
	attr0: string?,
	attr1: string?
};

CREATE DATASET RoadNetwork (RoadNetworkType) PRIMARY KEY id AUTOGENERATED;
CREATE DATASET Buildings (BuildingsType) PRIMARY KEY id AUTOGENERATED;
CREATE DATASET AllNodes (AllNodesType) PRIMARY KEY id AUTOGENERATED;
CREATE DATASET TempAllNodes (AllNodesType) PRIMARY KEY id AUTOGENERATED;
CREATE DATASET TempRoadNetwork (RoadNetworkType) PRIMARY KEY id AUTOGENERATED;
success
2a_Load_All.sql
USE TinyBenchmark;
Use TinyBenchmark;-- USE TinyBenchmark;

-- 'path'='127.0.0.1:///Users/andretra

success
37_Land-Information-Management_Average-SqFt-Buildings-States.sql
USE TinyBenchmark;
SELECT States.name, AVG(st_area(Buildings.g)) as AvgArea
FROM States, Buildings
WHERE st_contains(States.g, Buildings.g)
GROUP BY States.name
success
done


- uniqueID
- clientCopntextID as filename
- check for status if failed or success
- elapsed time
- timestamp

- create sample output file (json and csv)

- collect memory usage over time (timeseries)
    - peak memory (how tight a process is in terms of memory comsumption)

In [ ]:
# statement = "DROP DATAVERSE TinyBenchmark IF EXISTS"
# payload = {'statement': statement, 'pretty': True, 'client_context_id': 'test', 'readonly': False}
# response = requests.post('http://localhost:19002/query/service', params = payload)
# response.json()

{'requestID': '15c74eb0-865c-4842-9f48-07c2b7085539',
 'clientContextID': 'test',
 'signature': {'*': '*'},
 'plans': {},
 'status': 'success',
 'metrics': {'elapsedTime': '708.105908ms',
  'executionTime': '706.591289ms',
  'resultCount': 0,
  'resultSize': 0,
  'processedObjects': 0}}

In [14]:
# df = pd.read_csv('tinybenchmark.csv')
# df.dtypes

requestID           object
clientContextID     object
status              object
elapsedTime         object
resultCount          int64
resultSize           int64
processedObjects     int64
timestamp           object
dtype: object

In [35]:
# df['elapsedTime'] = df['elapsedTime'].str.replace('ms', '')
# df['elapsedTime'] = df['elapsedTime'].str.replace('s', '')
# df

,requestID,clientContextID,status,elapsedTime,resultCount,resultSize,processedObjects,timestamp
0,d5e629c6-6115-4196-86d2-cd6d31dee7f9,1_Create_All.sql,success,463.093025,0,0,0,2021-08-17 03:59:44.626400
1,bba2bf7e-2887-4aa5-bd74-ab21aafe8f56,2_Load_All.sql,success,5.913603398,0,0,0,2021-08-17 03:59:45.114684
2,08d88af8-221c-408b-836e-61859af120a3,3_Point-Count_Buildings.sql,success,117.550487,265,3988,265,2021-08-17 03:59:51.045120
3,c7733d79-22a4-4fbd-ba15-d85ef32f64ee,4_Point-Count_RoadNetwork.sql,success,255.429819,8415,126225,8415,2021-08-17 03:59:51.184122
4,0c101c5c-2acf-48bf-be0e-f08dc6c92f7d,5_Point-Count_AllNodes.sql,success,840.524768,47371,710565,47371,2021-08-17 03:59:51.509992
5,fb453cac-9c94-4f37-9443-0b98d5da007d,6_Equals_Polygon-Polygon.sql,success,1.358320914,265,343681,265,2021-08-17 03:59:52.710024
6,28fda66c-f554-4df5-ab1f-a01615ef9d15,7_Equals_Point-Point.sql,success,65.684378695,8345,5263399,8303,2021-08-17 03:59:54.731858
7,70f0bd6a-f027-4492-9d9c-2793fae5f90e,8_Disjoint_Polygon-Polygon.sql,success,8.716219244,69942,90702428,265,2021-08-17 04:01:01.881738
8,0391dab0-16bc-497b-9c65-84dbc9c12e2a,9_Intersects_Line-Polygon.sql,success,5.454665169,0,0,8680,2021-08-17 04:01:26.773352
9,55d119e8-9ec7-4476-bd87-d2c9dacca000,10_Intersects_Point-Polygon.sql,success,16.509145314,1399,1519244,47636,2021-08-17 04:01:45.995638
